In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Read the CSV file into a DataFrame
df = pd.read_csv('file_flights.csv', sep=';')

#Task1

# Sort the DataFrame by "airborne_lcl_tms" in ascending order (oldest to newest)
df['airborne_lcl_tms'] = pd.to_datetime(df['airborne_lcl_tms'])
df = df.sort_values(by=['acft_regs_cde', 'airborne_lcl_tms'])

# Add a new column "next_flight_info" that contains the information of the next flight
df['next_flight_info'] = df.groupby('acft_regs_cde')['id'].shift(-1)
df

#Task2
2. Build a dataset to know in every 15 minutes, how many flights coming in and out of an airport iata code in 2 hours before

# Load the CSV file
data = pd.read_csv('file_flights.csv', sep=';')

# Convert timestamp columns to datetime
data['actl_dep_lcl_tms'] = pd.to_datetime(data['actl_dep_lcl_tms'])
data['actl_arr_lcl_tms'] = pd.to_datetime(data['actl_arr_lcl_tms'])

# Define the airport IATA code you want to analyze
airport_iata_code = 'YVR'  # Change this to your desired airport code

# Create a DataFrame with timestamps at 15-minute intervals for the entire day
timestamps = pd.date_range(start='2022-12-31 00:00:00', end='2022-12-31 23:59:59', freq='15T')

# Initialize empty lists to store the counts
inbound_counts = []
outbound_counts = []

# Iterate through the 15-minute intervals
for ts in timestamps:
    # Count the number of inbound flights within the interval
    inbound_count = len(data[(data['actl_arr_lcl_tms'] >= ts) & (data['actl_arr_lcl_tms'] < ts + pd.Timedelta(minutes=15)) & (data['dest'] == airport_iata_code)])

    # Count the number of outbound flights within the interval
    outbound_count = len(data[(data['actl_dep_lcl_tms'] >= ts) & (data['actl_dep_lcl_tms'] < ts + pd.Timedelta(minutes=15)) & (data['orig'] == airport_iata_code)])

    inbound_counts.append(inbound_count)
    outbound_counts.append(outbound_count)

# Create a DataFrame to store the counts
count_data = pd.DataFrame({'Timestamp': timestamps, 'Inbound Flights': inbound_counts, 'Outbound Flights': outbound_counts})

count_data

#Task3
2. Build a dataset to know in every 15 minutes, how many flights coming in and out of an airport iata code in 2 hours before (all airport code)

# Convert date columns to datetime objects
df['actl_dep_lcl_tms'] = pd.to_datetime(df['actl_dep_lcl_tms'])
df['actl_arr_lcl_tms'] = pd.to_datetime(df['actl_arr_lcl_tms'])

# Initialize a list to store the results
results = []

# Define the time range for the dataset (from 00:00 to 23:59)
start_time = datetime.strptime('00:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('23:59:00', '%H:%M:%S').time()

# Initialize a timedelta for 15 minutes
time_interval = timedelta(minutes=15)

# Create a list of unique airport IATA codes
airport_iata_codes = df['orig'].append(df['dest']).unique()

# Generate timestamps in 15-minute intervals for the specified time range
current_time = start_time
while current_time <= end_time:
    # Calculate the end time of the 15-minute interval
    end_of_interval = (datetime.combine(datetime.today(), current_time) + time_interval).time()

    # Filter flights departing in the current 15-minute interval
    outgoing_flights = df[(df['actl_dep_lcl_tms'].dt.time >= current_time) & (df['actl_dep_lcl_tms'].dt.time < end_of_interval)]

    # Filter flights arriving in the current 15-minute interval
    incoming_flights = df[(df['actl_arr_lcl_tms'].dt.time >= current_time) & (df['actl_arr_lcl_tms'].dt.time < end_of_interval)]

    # Create a timestamp for the current 15-minute interval
    timestamp = pd.Timestamp('2022-12-31 ' + str(current_time))

    # Initialize a dictionary to store counts for each airport
    airport_counts = {airport: 0 for airport in airport_iata_codes}

    # Count the number of flights for each unique airport code
    for airport in airport_iata_codes:
        airport_counts[airport] += len(outgoing_flights[outgoing_flights['dest'] == airport])
        airport_counts[airport] += len(incoming_flights[incoming_flights['orig'] == airport])

    # Append the results to the list
    result_dict = {'tms': timestamp}
    result_dict.update(airport_counts)
    results.append(result_dict)

    # Move to the next 15-minute interval
    current_time = end_of_interval

# Create a DataFrame from the results
result_df = pd.DataFrame(results)
result_df